In [ ]:
if globals().get('LOADED_VIEWING_TABS') == None:
    display('LOADED_VIEWING_TABS')
    LOADED_VIEWING_TABS=True

In [ ]:
if globals().get('LOADED_ANNOTATIONS') == None:
    %run ANNOTATIONS.ipynb

In [ ]:
if globals().get('LOADED_PATTERN_OBSERVER') == None:
    %run PATTERN_OBSERVER.ipynb

In [ ]:
# %run DATAFRAME.ipynb

## matplotlib

In [ ]:
# imports
%matplotlib widget 

# plot
## a.plot(method='pandas.matplotlib')

## plotly

In [ ]:
# imports
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# plot
## a.plot(method='cufflinks') # WORKS

In [ ]:
# imports

# plot
## a.plot(method='pandas.plotly')

In [ ]:
# imports
import plotly.express as px

# plot
## a.plot(method='plotly_express')

## bokeh

In [ ]:
# imports
import pandas_bokeh
pandas_bokeh.output_notebook() # output in cell

# plot
## a.plot(method='pandas.pandas_bokeh')

In [ ]:
# imports

# plot
## a.plot(method='pandas.holoviews')

In [ ]:
'''
dat3.plot(backend='pandas_bokeh')
dat3.plot(backend='holoviews') # works outside of OUTPUT widget
'''
None

In [ ]:
from ipywidgets import Text,Output,GridspecLayout # for SINGLE_TAB
from ipywidgets import Tab,Layout # for TAB_COLLECTION

In [ ]:
'''
doesnt contain data
only explains how to generate a plot off a pd.DataFrame
'''
class SINGLE_PLOT:
    def __init__(self,
        name        : str              = 'SINGLE_PLOT',
        description : str              = 'SINGLE_PLOT_DESCRIPTION',
        query       : str              = None,
        df          : pd.DataFrame     = None,
        method      : str              = None,
        ):
        # attributes
        self._times_drawn = 0
        self.name         = name
        self.description  = name
        self._method      = method
        self._df          = df
        self._query       = query

        # build widget
        #self.widget       = Output(layout=Layout(height='300px',width='10.5%'))
        self.widget       = Output()
    
    '''
# should be same = matplotlib
a.plot(method='pandas.matplotlib') # works
        
# should be same = plotly
a.plot(method='cufflinks') # works
a.plot(method='pandas.plotly') # works, but cant suppress legend
a.plot(method='plotly_express') # works
        
# should be same = bokeh
a.plot(method='pandas.pandas_bokeh') # works
a.plot(method='pandas.holoviews')
    '''
    # clear and redraw
    def plot(self,method=None) -> None :
        # do nothing if no query provided
        if self._query is None: 
            return
        
        # ensure method populated
        if method is None: 
            method=self._method
            
        self._times_drawn+=1 # increment counter
        self.widget.clear_output() # clear image
        with self.widget:
            print(self.name,'_plot_method',':',method)
            if method in [
                'pandas.matplotlib',   # matplotlib # NEED `%matplotlib widget` MAGIC FOR IT TO WORK
                'pandas.plotly',       # plotly express
                'pandas.pandas_bokeh', # bokeh # BOKEH DOESNT WORK IN OUTPUT
                'pandas.holoviews',    # bokeh # BOKEH DOESNT WORK IN OUTPUT
                'pandas.altair',       # altair # DOESNT WORK
            ]:
                plotting_backend=method.split('.')[1]
                print('backend',plotting_backend)
                if plotting_backend in [
                    'matplotlib',
                ]:
                    out=self._df[self._query].unstack().plot(backend=plotting_backend,title=self._query,legend=False)
                    display(out)
                elif plotting_backend in [
                    'plotly',
                ]:
                    out=self._df[self._query].unstack().plot(backend=plotting_backend,title=self._query) # dont know how to suppress legend
                    display(out)
                elif plotting_backend in [
                    'pandas_bokeh', # bokeh
                ]:
                    out=self._df[self._query].unstack().plot(backend=plotting_backend,title=self._query,legend=False)
                    display(out)
                elif plotting_backend in [
                    'holoviews',    # bokeh
                ]:
                    out=self._df[self._query].unstack().plot(backend=plotting_backend,legend=False)
                    display(out)
                else:
                    print('nothing')
            elif method == 'cufflinks':
                self._df[self._query].unstack().iplot(title=self._query,legend=False)
            elif method == 'bokeh':
                    out=figure()
                    #out.plot(self._df[self._query].unstack().plot(backend=plotting_backend))
                    #show(out,notebook_handle=True)                
            elif method == 'plotly_express':
                out=px.line(
                    self._df[self._query].unstack(),
                    title=self._query
                )
                out.update_layout(showlegend=False)
                out.show()
            else:
                print('default')
                display(Text(self.name+'-'+str(self._query)+'-'+str(self._times_drawn)))

In [ ]:
# test case where no query provided
'''
a=SINGLE_PLOT('plotname')

display(a._method)
display(a._query)
display(a._times_drawn)
a.plot()
'''
None

In [ ]:
# example
'''

a=SINGLE_PLOT(
    name='SOME_PLOT',
    df=dat2,
    query='open',
    method='plotly_express'
)
display(a.widget)
a.plot()

a.plot() # default

a.plot(method='cufflinks') # cufflinks
'''
None

In [ ]:
'''
doesnt contain data
only explains how to plots within it generate plots off a pd.DataFrame
'''
class SINGLE_TAB:
    def __init__(self,
        name        : str              = 'SINGLE_TAB',
        description : str              = 'SINGLE_TAB_DESCRIPTION',
        cols        : int              = 3,
        rows        : int              = 3,
        method      : str              = 'plotly_express',
        df          : pd.DataFrame     = None,
        queries     : List[str]        = None,
        plots       : LOCATIONED_PLOTS = None,
        ):
        self._times_drawn = 0
        self.name         = name
        self.description  = description
        self._cols        = cols
        self._rows        = rows
        self._method      = method
        self._df          = df
        self._queries     = queries
        self._plots       = plots if not plots==None else self._plots_from_queries() # populate default
        self.widget       = self._make()

    ####################################
    # populate defaults
    ####################################
    def _plots_from_queries(self):
        # populate queries, assumes self._queries is shorter than self._rows*self._cols
        queries=[None]*self._rows*self._cols
        if self._queries != None:
            queries[:len(self._queries)]=self._queries

        # generate single plots from queries
        idx=0
        out={}
        for r in range(self._rows):
            for c in range(self._cols):
                out[(r,c)]=SINGLE_PLOT(
                    name=self.name+'['+str(r)+str(c)+']',
                    method=self._method,
                    df=self._df,
                    query=queries[idx],
                )
                idx+=1
        return out

    def _make(self) -> GridspecLayout :
        # instantiate
        gridspec=GridspecLayout(n_rows=self._rows,n_columns=self._cols)
        
        # populate gridspec
        for r in range(self._rows):
            for c in range(self._cols):
                gridspec[r,c]=self._plots[(r,c)].widget
        
        # return
        return gridspec
    
    # clear and draw with given data
    def update_tab(self,method=None) -> None :
        # ensure method populated
        if method is None: 
            method=self._method

        self._times_drawn+=1
        for x in self._plots.values():
            x.plot(method)
        #print(self.name,'update_tab',self._times_drawn)

In [ ]:
# example
'''

a=SINGLE_TAB(df=dat2,queries=['open','high','low','close','adjusted_close'])
a.widget

a.update_tab()

a.update_tab(method='pandas.plotly')

a.update_tab(method='pandas.matplotlib')

a.update_tab(method='cufflinks')

'''
None

In [ ]:
'''
doesnt contain data
only explains how to tabs within it generate plots off a pd.DataFrame
'''
class TAB_COLLECTION(IObserver):
    def __init__(self,
        name        : str          = 'TAB_COLLECTION',
        description : str          = 'TAB_COLLECTION_DESCRIPTION',
        tabs        : TABS         = None,
        df          : pd.DataFrame = None,
        ):
        self.name=name
        self.description=description
        
        self._tabs=tabs if tabs!=None else self._populate_defaults_tabs() # contents of tab collection
        self._is_up_to_date=[True]*len(self._tabs) # nothing selected by default, all tabs up to date by default
        
        self.widget=self._make()
        self._observers=set() # for observer pattern

    ####################################
    # populate defaults
    ####################################
    def _populate_defaults_tabs(self):
        return [
            SINGLE_TAB('tab1'),
            SINGLE_TAB('tab2'),
            SINGLE_TAB('tab3'),
            SINGLE_TAB('tab4'),
        ]
    
    ####################################
    # observer pattern
    ####################################
    # observer
    def react(self,
        subject_name : str,
        subject_info : object
        ) -> None :
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_name',subject_name)
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_info',subject_info)
        
        if subject_name=='DATA_CONTAINER':
            self.delayed_full_update(subject_info['df'])

    ####################################
    # buttons
    ####################################
    '''change active tab, update as needed'''
    def _onchange_tab(self,change) -> None :
        print(self.name,':','CLICK','NAME[',self._active_tab.name,'] INDEX[',self._active_tab_index,']')

        # handle delayed update
        if not self._is_up_to_date[self._active_tab_index]:
            print(self.name,':','UPDATE[DELAYED]','NAME[',self._active_tab.name,'] INDEX[',self._active_tab_index,']')
            self._active_tab.update_tab(self._data) # update this tab
            self._is_up_to_date[self._active_tab_index]=True # flag tab as up to date
            
        # logging
        self._logging()
        
    ####################################
    # GUI
    ####################################
    '''build tab collection'''
    def _make(self) -> None :
        # make tab collection
        out=Tab(
            children=[x.widget for x in self._tabs],
            layout=Layout(width='100%',height='auto',min_height='500px')
        )

        # name tabs
        for i,tab_name in enumerate(x.name for x in self._tabs):
            out.set_title(index=i,title=tab_name)
            
        # observe active tab
        out.observe(self._onchange_tab,names='selected_index')

        # return
        return out

    '''delayed update of tab collection, update active tab immediate, delay update of non-active tabs till when they are viewed'''
    def delayed_full_update(self,data : pd.DataFrame) -> None:
        # only update active tab, delay update of inactive tabs till they become active
        for i,tab in enumerate(self._tabs):
            if self._active_tab_index==i:
                print(self.name,':','UPDATE[INSTANT]','NAME[',self._active_tab.name,'] INDEX[',self._active_tab_index,']')
                tab.update_tab(data) # update this particular tab
                self._is_up_to_date[i]=True # flag is up to do date
            else:
                self._is_up_to_date[i]=False # flag these tabs as out of date, update when viewed
            
        # logging
        self._logging()
    
    @property
    def _active_tab_index(self) -> int :
        return self.widget.selected_index
    
    @property
    def _active_tab(self) -> str :
        return self._tabs[self._active_tab_index]
    
    @property
    def _times_drawn(self) -> List[int]:
        return [x._times_drawn for x in self._tabs]
    
    def _logging(self) -> None:
        # delayed updating
        print(self.name,':','compute times',self._times_drawn)
        print(self.name,':','is up to date',self._is_up_to_date)

# example
TAB_COLLECTION().widget

In [ ]:
'''
# example
a=TAB_COLLECTION(tabs=
    [
        SINGLE_TAB('a'),
        SINGLE_TAB('b'),
        SINGLE_TAB('c'),
        SINGLE_TAB('d'),
    ]
)
a.widget

# global delayed update
a.delayed_full_update('abc')

# request immediate update of specific tab
a._tabs[0].update_tab('asdf')

a._tabs[0].update_tab('1')

a._tabs[2].update_tab('2')

a._tabs[2].update_tab('3')

a._tabs[3].update_tab('4')

a._times_drawn
'''
None